In [ ]:
from datetime import datetime, timedelta
import importlib
from dasilva_invariants import diagnostics, meshes, invariants
import numpy as np
from joblib import delayed, Parallel
import pyvista
import pylab as plt
from matplotlib.colors import LogNorm
import seaborn as sns


importlib.reload(diagnostics)
importlib.reload(meshes)
importlib.reload(invariants)

%matplotlib inline

In [ ]:
sns.set_style('darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
DPI = 500

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def task_target(mesh_fname, r, mlat, i, j):
    mesh = pyvista.read(mesh_fname)
    try:
        res = invariants.calculate_K(mesh, (r, 0, 0), mlat)
    except invariants.FieldLineTraceReturnedEmpty:
        return np.nan, np.nan, np.nan, i, j

    ii = res.trace_latitude.searchsorted(np.deg2rad(mlat))
    if ii == res.trace_points.shape[0]:
        return np.nan, np.nan, np.nan, i, j

    xout = res.trace_points[ii, 0]
    zout = res.trace_points[ii, 2]

    return xout, zout, res.K, i, j

In [ ]:
def process(mesh, nmlats=100, nrvalues=100, tail_rmax=7):
    # Do processing in parallel ---------------------------------------------
    lower_rvalue = np.linalg.norm(mesh.points, axis=1).min()
    mlats = np.linspace(-75, 75, nmlats)
    rvalues = []
    rvalues.extend((-np.linspace(lower_rvalue, tail_rmax, nrvalues//2)).tolist()[::-1])
    rvalues.extend(np.linspace(lower_rvalue, 7, nrvalues//2).tolist())
    rvalues = np.array(rvalues)

    mesh_fname = 'tmp.vtk'
    mesh.save(mesh_fname)

    tasks = []

    for i, mlat in enumerate(mlats):
        for j, r in enumerate(rvalues):
            tasks.append(delayed(task_target)(mesh_fname, r, mlat, i, j))

    print(f'{len(tasks)} tasks total')
    p = Parallel(verbose=10000, backend='multiprocessing', n_jobs=-1)
    parallel_results = p(tasks)

    # Convert to numpy arrays -----------------------------------------------
    X = np.zeros((mlats.size, rvalues.size), dtype=float)
    Z = np.zeros((mlats.size, rvalues.size), dtype=float)
    K = np.zeros((mlats.size, rvalues.size), dtype=float)

    for xout, zout, Kout, i, j in parallel_results:
        X[i, j] = xout
        Z[i, j] = zout
        K[i, j] = Kout

    return X, Z, K

In [ ]:
def plot(mesh, X, Z, K, title):
    K = K.copy()
    K[K == 0] = K[K > 0].min()

    plt.figure(figsize=(16, 9))
    plt.contour(X, Z, np.log10(K), levels=np.arange(-5, 1, .5), colors='black')
    plt.pcolor(X, Z, K, norm=LogNorm(vmin=5e-7, vmax=5))

    plt.title(title, fontweight='bold')
    plt.colorbar().set_label('K ($Re\sqrt{G}$)')
    plt.xlabel('X SM (Re)')
    plt.ylabel('Z SM (Re)')
    plt.xlim(7.5, -7.5)
    plt.ylim(-6, 6)
    plt.grid(True, color='#ccc', linestyle='dashed')

    circ = plt.Circle((0, 0), 1, color='#4470AD')
    plt.gca().add_patch(circ)

    for r in np.arange(3, 8):
        for th in [0, 180]:
            x, y = np.cos(np.deg2rad(th)) * r, np.sin(np.deg2rad(th)) * r
            try:
                res = invariants.calculate_K(mesh, (x, y, 0), 7.5)
            except invariants.FieldLineTraceReturnedEmpty:
                continue
            plt.plot(res.trace_points[:, 0],
                     res.trace_points[:, 2],
                     color='k')

In [ ]:
lfm_path = '/glade/u/home/danieldas/scratch//data/LFM-20131002_RBSP/ElkStorm_mhd_2013-10-02T07-30-00Z.hdf'
lfmrcm_path = '/glade/u/home/danieldas/scratch//data/LFMRCM-20131002_RBSP/ElkStorm-LR_mhd_2013-10-02T07-30-00Z.hdf'

!ls $lfm_path
!ls $lfmrcm_path

In [ ]:
importlib.reload(invariants)

lfm_mesh = meshes.get_lfm_hdf4_data(lfm_path)
lfm_results = process(lfm_mesh)

In [ ]:
plot(lfm_mesh, *lfm_results, 'LFM During Recovery Phase')

In [ ]:
importlib.reload(invariants)

lfmrcm_mesh = meshes.get_lfm_hdf4_data(lfmrcm_path)
lfmrcm_results = process(lfmrcm_mesh)

In [ ]:
plot(lfmrcm_mesh, *lfmrcm_results, 'LFM-RCM During Recovery Phase')

In [ ]:
Vsw = 591.5 
dist_L1_earth = 1467000 #https://www.johndcook.com/blog/2021/12/28/lagrange-points-l1-and-l2/
delay_sec = dist_L1_earth / Vsw
delay_sec/60
time = datetime(2013, 10, 2, 7, 30) + timedelta(seconds=delay_sec)

In [ ]:
t96_mesh = meshes.get_tsyganenko_on_lfm_grid_with_auto_params(
    'T96', time, lfm_path, tell_params=True, force_zero_tilt=False,
)

In [ ]:
t04_mesh = meshes.get_tsyganenko_on_lfm_grid_with_auto_params(
    'T04', time, lfm_path, tell_params=True, force_zero_tilt=False,
)

In [ ]:
t96_results = process(t96_mesh)

In [ ]:
t04_results = process(t04_mesh)

In [ ]:
plot(t96_mesh, *t96_results, 'T96 During Recovery Phase')

In [ ]:
plot(t04_mesh, *t04_results, 'T05 During Recovery Phase')